In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Data Load

In [14]:
sub_path = '../input/statoil-iceberg-submissions'
all_files = os.listdir(sub_path)
all_files = all_files[1:5]
all_files.remove('sub_200_ens_densenet.csv')
all_files.remove('sub_keras_beginner.csv')
all_files.append('submission38.csv')
all_files.append('submission43.csv')
all_files

['sub_fcn.csv', 'sub_TF_keras.csv', 'submission38.csv', 'submission43.csv']

In [15]:
out1 = pd.read_csv('../input/statoil-iceberg-submissions/sub_200_ens_densenet.csv', index_col=0)
out2 = pd.read_csv('../input/statoil-iceberg-submissions/sub_TF_keras.csv',index_col=0)
out3 = pd.read_csv('../input/submission38-lb01448/submission38.csv',index_col=0)
out4 = pd.read_csv('../input/submission38-lb01448/submission43.csv',index_col=0)
concat_sub = pd.concat([out1,out2,out3,out4], axis=1)
cols = list(map(lambda x:'is_iceberg_' + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace= True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,1.943216e-02,0.005586,0.008875,0.005586
1,4023181e,3.168809e-02,0.145927,0.146779,0.031688
2,b20200e4,4.000000e-08,0.000015,0.048646,0.001000
3,e7f018bb,9.925741e-01,0.999914,0.999184,0.999000
4,4371c8c3,2.215107e-02,0.033843,0.277251,0.022151


In [16]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.890434,0.986291,0.994880
is_iceberg_1,0.890434,1.000000,0.923848,0.900871
is_iceberg_2,0.986291,0.923848,1.000000,0.988002
is_iceberg_3,0.994880,0.900871,0.988002,1.000000


In [17]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:,1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:,1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:,1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:,1:6].median(axis=1)

In [18]:
# set up cutoff threshold for lower and upper bounds, easy to twist
cutoff_lo = 0.8
cutoff_hi = 0.2

# Mean Stacking

In [19]:
#concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
#concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', index=False, float_format='%.6f')

# Median Stacking

In [20]:
#concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
#concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', index=False, float_format='%.6f')

# PushOut + Median Stacking

In [21]:
#concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
#                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
#                                             0, concat_sub['is_iceberg_median']))
#concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
#                                        index=False, float_format='%.6f')

# MinMax + Mean Stacking

In [22]:
#concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
#                                    concat_sub['is_iceberg_max'], 
#                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
#                                             concat_sub['is_iceberg_min'], 
#                                             concat_sub['is_iceberg_mean']))
#concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
#                                        index=False, float_format='%.6f')

# MinMax + Median Stacking

In [23]:
#concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
#                                    concat_sub['is_iceberg_max'], 
#                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
#                                             concat_sub['is_iceberg_min'], 
#                                             concat_sub['is_iceberg_median']))
#concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
#concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
#                                       index=False, float_format='%.6f')

# MinMax + BestBase Stacking

In [24]:
# load the model with best base performance
sub_base = pd.read_csv('../input/submission38-lb01448/submission43.csv')

In [25]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:4] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:4] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('submission54.csv', 
                                        index=False, float_format='%.6f')